In [1]:
import numpy as np
import sys
import itertools as itertools
sys.path.append("..")
sys.path.append("../..")

import matplotlib.pyplot as plt
from python.Hubbard import Hubbard_model_with_filling, Double_occupancy, get_filling
from python.DMRG import DMRG
from python.Zippers import MPO_to_Hamiltonian, contract_MPS
from python.Canonical_Form import get_Neumann_entropy
from python.Backend import Backend

In [2]:
n_sites = 50
hopping_t = 1.0

In [3]:
device = "numpy"
# device = "torch"

bk = Backend(device)

NKeep = 50
NSweep = 20
Krylov_bases = 5 #* 5 is usually enough
Lanczos_cutoff = 1e-2
iterative_diag = True
two_site = True
verbose = True

In [4]:
import numpy.typing as npt
import pickle
from dataclasses import dataclass, field, asdict, replace
from copy import deepcopy
from typing import Any, Optional
from pathlib import Path


@dataclass
class Params:
    L: int
    t: float
    U: float
    mu: float
    NKeep: int
    NSweep: int
    diag: bool
    Krylov: int
    cutoff: float
    two: bool
    
    def to_log(self) -> str:
        log_dict = asdict(self)
        
        log_dict['t'] = round(self.t, 2)
        log_dict['U'] = round(self.U, 2)
        log_dict['mu'] = round(self.mu, 2)
        log_dict['cutoff'] = round(self.cutoff, 3)
        
        # Filter out keys where the value is None
        filtered_log_dict = {key: log for key, log in log_dict.items() if log is not None}
        
        # Create log string
        return "_".join(
            f"{key}={log}" for key, log in filtered_log_dict.items()
        )

@dataclass
class Results:
    ground_MPS: list[npt.NDArray]
    ground_state_energy: float
    Neumann_entropy: float
    filling: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    double_occupancy: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    ground_energies: list[float] | npt.NDArray[np.float64] = field(default_factory=list)
    
    def to_array(self):
        return replace(
            self, **{field: np.array(getattr(self, field))
            if isinstance(getattr(self, field), list) else getattr(self, field)
            for field in asdict(self)}
        )
        
    def self_destruct(self):
        for field_name in self.__dataclass_fields__:
            field_value = getattr(self, field_name)
            if isinstance(field_value, list):
                field_value.clear()  # Clear list contents
            else:
                setattr(self, field_name, None)  # Set to None if not a list



In [5]:
U_start = 9
U_end = 10
delta_U = 1
delta_mu = 0.1

mu_margin = 2.5

In [ ]:
interaction_Us = np.arange(U_start + delta_U, U_end + delta_U, delta_U)

for it1, interaction_U in enumerate(interaction_Us):
    
    chemical_potentials = np.arange(-mu_margin, mu_margin+interaction_U+delta_mu, delta_mu)
    
    for it2, chemical_potential in enumerate(chemical_potentials):
        
        if verbose:
            print(f"t={round(hopping_t,2)} | U={round(interaction_U,2)} | mu={round(chemical_potential,2)}")
        
        Hubbard_Hamiltonian = Hubbard_model_with_filling(
            bk = bk,
            n_sites=n_sites, hopping_t=hopping_t,
            interaction_U=interaction_U, chemical_potential=chemical_potential,
        )
        
        ground_energies, _, ground_MPS = DMRG(
            bk = bk,
            Hamiltonian = Hubbard_Hamiltonian,
            NKeep = NKeep,
            NSweep = NSweep,
            Krylov_bases = Krylov_bases,
            Lanczos_cutoff = Lanczos_cutoff,
            iterative_diag = iterative_diag,
            two_site = two_site,
            verbose = verbose,
        )
        
        param = Params(
            L = n_sites,
            t = hopping_t,
            U = interaction_U,
            mu = chemical_potential,
            NKeep = NKeep,
            NSweep = NSweep,
            diag = iterative_diag,
            Krylov = Krylov_bases,
            cutoff = Lanczos_cutoff,
            two = two_site
        )
        
        filename = f"{param.to_log()}"
        dir_name = Path(f"{device}/data")
        dir_name.mkdir(parents=True, exist_ok=True)
        
        filling = get_filling(ground_MPS, bk=bk)
        double_occupancy = Double_occupancy(ground_MPS, bk=bk)
        ground_state_energy = ground_energies[-1]
        Neumann_entropy = get_Neumann_entropy(ground_MPS, bk=bk)
        
        result = Results(
            ground_MPS = [],
            filling = filling,
            double_occupancy = double_occupancy,
            ground_state_energy = ground_state_energy,
            Neumann_entropy = Neumann_entropy,
            ground_energies = ground_energies,
        )
        
        output: dict[str, Any] = {}
        output.update(asdict(param))
        output.update(asdict(result))
        
        with open(dir_name / f"{filename}.pkl", "wb") as file:
            pickle.dump(output, file)
        
        # except Exception as e:
        #     print(e)
        
        

t=1.0 | U=10 | mu=-2.5
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=0.0 | time=0js
iter=1 | energy=0j | time=0.5555102910002461s
t=1.0 | U=10 | mu=-2.4
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=0.0 | time=0js
iter=1 | energy=0j | time=0.38704705700001796s
t=1.0 | U=10 | mu=-2.3
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=0.0 | time=0js
iter=1 | energy=0j | time=0.6634877339993182s
t=1.0 | U=10 | mu=-2.2
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=0.0 | time=0js
iter=1 | energy=0j | time=0.4852294239999537s
t=1.0 | U=10 | mu=-2.1
L=50 | NKeep=50 | NSweep=20 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=0.0 |